# Interactive visualization

In [ ]:
import os
import pandas as pd
import numpy as np
import json
import folium
from IPython.display import display, HTML
from ipywidgets import widgets
from ipywidgets import interact
import matplotlib.pyplot as plt
import seaborn as sns
from pylab import rcParams

# European unemployment rates
The dataset for european unemployment rates can be found on [eurostat](http://ec.europa.eu/eurostat/data/database) by using the search bar and looking for the term "Unemployment rate - annual data". The internal code for the dataset is `tipsun20`. We download it using the .xls format because it is much cleaner than the .tsv version. We added an external preprocessing step with Excel where we simply deleted the textual descriptions in the file and unnecessary columns. This will lead to a cleaner code for loading the data. There is also subtelty that arises when plotting the choropleth map. If instead of the xls file we had taken the tsv format, we would have received data with two-letters country indicators (such as FR for France). However, the codes used by eurostat do not match the codes used in the topojson file. Therefore, taking the xls format has the additional advantage to enumerate countries by their full names which are a better match to our topojson file.

In [ ]:
europe_unemployment_data_2016 = pd.read_csv('data/europe_unemployment.csv', names=['Country', 'Rate'], header=0)
europe_unemployment_data_2016.head()

Before creating the choropleth map, let us look at the distribution of unemployment rates across european countries:

In [ ]:
europe_unemployment_data_2016.plot.hist(cumulative=True)
europe_unemployment_data_2016.plot.hist()
plt.show()

Because the threshold by default of the choropleth maps is linear (even though the documentation says it is not), we decided the tune it according to the above histograms to better visualise the difference between european countries. For this, we will use the following quantiles:

In [ ]:
print(europe_unemployment_data_2016.Rate.quantile([0, .1, .25, .5, .9, 1.]))

We then load the TopoJson file. Note that Folium will still show geometries contained in the TopoJson which do not have any data associated to them. This can be very misleading as it may suggest that the countries for which we do not have any data have actually a low unemployment rate. This is we perform a quick filtering operation on the TopoJson file to remove any country that is not present in our unemployment dataset.

In [ ]:
europe_json = json.load(open("topojson/europe.topojson.json"))

europe_json['objects']['europe']['geometries'] = \
                        [
                            x
                            for x in europe_json['objects']['europe']['geometries'] 
                            if (x['properties']['NAME'] in europe_unemployment_data_2016['Country'].values)
                        ]

We can finally display the map:

In [ ]:
m = folium.Map([47.870833, 9.085721], tiles='Mapbox Bright', zoom_start=4)
m.choropleth(europe_json, 
             data=europe_unemployment_data_2016, 
             columns=['Country', 'Rate'], 
             key_on='feature.properties.NAME', # Joining on names
             fill_color='YlOrRd',
             topojson='objects.europe',
             line_color='gray',
             threshold_scale=list(europe_unemployment_data_2016.Rate.quantile([0, .10, .25, .5, .9, 1.]).values)
             )
m

We will see that the Swiss unemployment rate is very low in comparison to european countries. In fact, it is one of the lowest.

# 2 Swiss unemployment data
## 2.a Jobless people rate

In [ ]:
swiss_unemployment_data = pd.read_csv('data/swiss_unemployment_total.csv')
swiss_unemployment_rates = swiss_unemployment_data.copy()[['Canton','Taux de chômage (1 cav)','Taux de chômage des jeunes (1 pdv)']]
swiss_unemployment_rates.head()

In [ ]:
cantons_dict = {'Zurich': 'ZH', 'Berne': 'BE','Lucerne':'LU','Uri':'UR','Schwyz':'SZ',
    'Obwald':'OW','Nidwald':'NW','Glaris':'GL','Zoug':'ZG','Fribourg':'FR',
    'Soleure':'SO','Bâle-Ville':'BS','Bâle-Campagne':'BL','Schaffhouse':'SH',
    'Appenzell Rhodes-Extérieures':'AR','Appenzell Rhodes-Intérieures':'AI',
    'St-Gall':'SG','Grisons':'GR','Argovie':'AG','Thurgovie':'TG','Tessin':'TI',
    'Vaud':'VD','Valais':'VS','Neuchâtel':'NE','Genève':'GE','Jura':'JU'}

swiss_unemployment_rates.Canton.replace(cantons_dict, inplace=True)

In [ ]:
swiss_json = json.load(open("topojson/ch-cantons.topojson.json"))

map_ = folium.Map([46.819393, 8.333115], tiles='CartoDBpositron', zoom_start=8)
map_.choropleth(swiss_json, 
             data=swiss_unemployment_rates, 
             columns=['Canton', 'Taux de chômage (1 cav)'], 
             key_on='feature.id', 
             fill_color='YlOrRd',
             topojson='objects.cantons',
             line_color='gray'
             )
map_

## 2.b Alternative definition of unemployment: jobseekers rate

Now we also consider in our rates the number of people that currently have a job but are also registered in a placement office. Because we only have totals available for this variable instead of rates, we need to compute it ourselves. To this end, we will load a different dataset. It contains the total number of job seekers without a job as well as job seekers with a job. We will use the former to find back the number of active people per canton and finally, compute the rates of job seekers with a current job.

In [ ]:
swiss_unemployment_numbers_data = pd.read_csv('data/swiss_unemployed_and_job_seekers.csv')
swiss_unemployment_numbers = swiss_unemployment_numbers_data.copy()
swiss_unemployment_numbers.head(3)



We extract the two variables of interest: "Demandeurs d'emploi" and "Chômeurs inscrits"


In [ ]:
chomeurs_mask = swiss_unemployment_numbers.columns.str.endswith("2016")
demandeurs_mask = swiss_unemployment_numbers.columns.str.endswith("2016.1")

swiss_unemployment_numbers_chomeurs = pd.concat([
    swiss_unemployment_numbers['Canton'],
    swiss_unemployment_numbers.iloc[:, chomeurs_mask]
], axis=1)[1:]

swiss_unemployment_numbers_demandeurs = pd.concat([
    swiss_unemployment_numbers['Canton'],
    swiss_unemployment_numbers.iloc[:, demandeurs_mask]
], axis=1)[1:]

We take the average over the year for each canton:

In [ ]:
# Clean values and convert to numeric type
swiss_unemployment_numbers_chomeurs.replace("'", '', inplace=True, regex=True)
swiss_unemployment_numbers_demandeurs.replace("'", '', inplace=True, regex=True)
swiss_unemployment_numbers_chomeurs = swiss_unemployment_numbers_chomeurs.apply(pd.to_numeric, errors='ignore')
swiss_unemployment_numbers_demandeurs = swiss_unemployment_numbers_demandeurs.apply(pd.to_numeric, errors='ignore')
      
# Compute average
swiss_unemployment_numbers_chomeurs['AVG'] = swiss_unemployment_numbers_chomeurs.mean(axis=1)
swiss_unemployment_numbers_demandeurs['AVG'] = swiss_unemployment_numbers_demandeurs.mean(axis=1)

# Keep only cantons and average
swiss_unemployment_numbers_chomeurs = swiss_unemployment_numbers_chomeurs[['Canton', 'AVG']]
swiss_unemployment_numbers_demandeurs = swiss_unemployment_numbers_demandeurs[['Canton', 'AVG']]

# Convert cantons to two-letters code
swiss_unemployment_numbers_chomeurs.Canton.replace(cantons_dict, inplace=True)
swiss_unemployment_numbers_demandeurs.Canton.replace(cantons_dict, inplace=True)

# Remove totals
swiss_unemployment_numbers_chomeurs = swiss_unemployment_numbers_chomeurs[swiss_unemployment_numbers_chomeurs['Canton'] != 'Total']
swiss_unemployment_numbers_demandeurs = swiss_unemployment_numbers_demandeurs[swiss_unemployment_numbers_demandeurs['Canton'] != 'Total']

print(swiss_unemployment_numbers_chomeurs.head(3))
print(swiss_unemployment_numbers_demandeurs.head(3))



We can now compute the active population by canton using the dataframe from the previous question. We have:
$$ \textrm{unemployment rate}= \frac{\textrm{unemployed people}}{\textrm{active population}} \Leftrightarrow \textrm{active population}= \frac{\textrm{unemployed people}}{\textrm{unemployment rate}}$$


In [ ]:
# Sort by canton to align the data
swiss_unemployment_rates_sorted = swiss_unemployment_rates.sort_values('Canton').reset_index(drop=True)
swiss_unemployment_numbers_chomeurs_sorted = swiss_unemployment_numbers_chomeurs.sort_values('Canton').reset_index(drop=True)

# Convert rates to numeric
swiss_unemployment_rates_sorted = swiss_unemployment_rates_sorted.apply(pd.to_numeric, errors='ignore')

swiss_active_population = pd.DataFrame(100.0 * swiss_unemployment_numbers_chomeurs_sorted['AVG'] / swiss_unemployment_rates_sorted['Taux de chômage (1 cav)'], columns=['Total'])
print(swiss_active_population.head())



We can finally obtain the rates of job seeker per canton:


In [ ]:
# Align the job seeker data
swiss_unemployment_numbers_demandeurs_sorted = swiss_unemployment_numbers_demandeurs.sort_values('Canton').reset_index(drop=True)
swiss_unemployment_numbers_demandeurs_sorted['Taux'] = 100.0 * swiss_unemployment_numbers_demandeurs_sorted['AVG'] / swiss_active_population['Total']
print(swiss_unemployment_numbers_demandeurs_sorted.head())



With this definition the rates are typically higher. This was to be expected since by definition, unemployed people are a subset of job seekers. Although the distribution changed a bit, the main trend is still present. That is, the cantons with the highest rates are the french and italian speaking ones.


In [ ]:
map_ = folium.Map([46.819393, 8.333115], tiles='CartoDBpositron', zoom_start=8)
map_.choropleth(swiss_json, 
             data=swiss_unemployment_numbers_demandeurs_sorted, 
             columns=['Canton', 'Taux'], 
             key_on='feature.id', 
             fill_color='YlOrRd',
             topojson='objects.cantons',
             line_color='gray'
             )
map_

# 3 Swiss unemployment, split

## 3.a Split by nationality
We load a new dataset downloaded from amstat. This time, the datasets distinguishes the unemployment rates of Swiss natives and foreigner.

In [ ]:
swiss_unemployment_nationality_data = pd.read_csv('data/swiss_unemployment_nationality_split.csv')
swiss_unemployment_nationality_rates = swiss_unemployment_nationality_data.copy()[['Canton','Taux de chômage (1 cav)','Nationalité']]

# Replace the cantons names by their two-letters code
swiss_unemployment_nationality_rates.Canton.replace(cantons_dict, inplace=True)

In [ ]:
# Extract the relevant data
swiss_unemployment_foreign_rates = swiss_unemployment_nationality_rates[swiss_unemployment_nationality_rates['Nationalité'] =='Etrangers']
swiss_unemployment_swiss_rates = swiss_unemployment_nationality_rates[swiss_unemployment_nationality_rates['Nationalité'] =='Suisses']



This time we plot two choropleth layers and add a LayerControl object. This allows to easily switch between the visualisation of unemployment rates for Swiss natives and unemployment rates for foreigners.


In [ ]:
map_ = folium.Map([46.819393, 8.333115], tiles='CartoDBpositron', zoom_start=8)
map_.choropleth(swiss_json, 
             data=swiss_unemployment_rates, 
             columns=['Canton', 'Taux de chômage (1 cav)'], 
             key_on='feature.id', 
             fill_color='YlOrRd',
             topojson='objects.cantons',
             name="Swiss natives unemployment rates",
             legend_name="Swiss natives unemployment rates",
             line_color='gray'
             )
map_.choropleth(swiss_json, 
             data=swiss_unemployment_foreign_rates, 
             columns=['Canton', 'Taux de chômage (1 cav)'], 
             key_on='feature.id', 
             fill_color='YlGn',
             topojson='objects.cantons',
             name="Foreigners unemployment rates",
             legend_name="Foreigners unemployment rates",
                line_color='gray'
             )
folium.LayerControl(collapsed=False).add_to(map_)
map_

In [ ]:
#TODO: comment results

## 3.b Split by nationality and age

In [ ]:
def to_numeric(x):
    return(x.str.replace('\'', '')).astype('int64')

In [ ]:
swiss_unemployment_nationality_age_data = pd.read_csv('data/swiss_unemployment_nationality_age_split.csv')
swiss_unemployment_nationality_age_rates = swiss_unemployment_nationality_age_data.copy()

swiss_unemployment_nationality_age_rates = swiss_unemployment_nationality_age_rates.drop(['Unnamed: 3','Mois'], 1)
swiss_unemployment_nationality_age_rates.loc[:,"Janvier 2016":"Décembre 2016"] = swiss_unemployment_nationality_age_rates.loc[:,"Janvier 2016":"Décembre 2016"].apply(lambda x: to_numeric(x), axis=1)
swiss_unemployment_nationality_age_rates["Mean"] = (swiss_unemployment_nationality_age_rates.loc[:,"Janvier 2016":"Décembre 2016"].sum(axis=1)/12).astype('int64')

swiss_unemployment_nationality_age_rates = swiss_unemployment_nationality_age_rates[['Canton','Nationalité','Classes d\'âge 15-24, 25-49, 50 ans et plus','Mean']]

swiss_unemployment_nationality_age_rates = swiss_unemployment_nationality_age_rates[swiss_unemployment_nationality_age_rates['Canton'] != 'Total']
swiss_unemployment_nationality_age_rates = swiss_unemployment_nationality_age_rates[swiss_unemployment_nationality_age_rates['Nationalité'] != 'Total']
swiss_unemployment_nationality_age_rates = swiss_unemployment_nationality_age_rates[swiss_unemployment_nationality_age_rates['Classes d\'âge 15-24, 25-49, 50 ans et plus'] != 'Total']

swiss_unemployment_nationality_age_rates['Nationality and age'] = swiss_unemployment_nationality_age_rates['Nationalité'] + swiss_unemployment_nationality_age_rates['Classes d\'âge 15-24, 25-49, 50 ans et plus'] 

swiss_unemployment_nationality_age_rates = swiss_unemployment_nationality_age_rates.pivot(index='Canton', columns= 'Nationality and age', values='Mean')

display(swiss_unemployment_nationality_age_rates)



In [ ]:
swiss_unemployment_nationality_age_rates.plot.barh(grid =False,fontsize= 30,width = 0.85)
#rcParams['figure.figsize'] = 40,40
plt.xlabel('Rate',fontsize = 45)
plt.ylabel('Canton', fontsize=45,rotation = 0)
plt.legend(
   fontsize=40)
plt.show()

# 4  Röstigraben Delimitation

In [ ]:
map_ = folium.Map([46.919393, 8.334115], tiles='cartodbpositron', zoom_start=8)

unemployment_swiss_dict = swiss_unemployment_swiss_rates.set_index('Canton')['Taux de chômage (1 cav)']
unemployment_foreign_dict = swiss_unemployment_foreign_rates.set_index('Canton')['Taux de chômage (1 cav)']

foreign_mean = unemployment_foreign_dict.mean()
swiss_mean = unemployment_swiss_dict.mean()

foreign_min = unemployment_foreign_dict.min()
swiss_min = unemployment_swiss_dict.min()

foreign_max = unemployment_foreign_dict.max()
swiss_max = unemployment_swiss_dict.max()


unem_swiss_unem_foreign_color = '#ae017e'
unem_swiss_em_foreign_color = '#fc4e2a'
em_swiss_unem_foreign_color = '#1d91c0'
em_swiss_em_foreign_color = '#fde0dd'



def my_color_function(feature):
    if unemployment_swiss_dict[feature['id']] > swiss_mean:
        if unemployment_foreign_dict[feature['id']] > foreign_mean:
            return unem_swiss_unem_foreign_color
        else:
            return unem_swiss_em_foreign_color
    else:
        if unemployment_foreign_dict[feature['id']] > foreign_mean:
            return em_swiss_unem_foreign_color
        else:
            return em_swiss_em_foreign_color
    

folium.TopoJson(
    swiss_json,
    object_path ='objects.cantons',
    style_function=lambda feature: {
        'fillColor': my_color_function(feature),
    }
).add_to(map_)

step = cm.StepColormap(
    [em_swiss_em_foreign_color, unem_swiss_em_foreign_color],
   vmin= swiss_min,vmax = swiss_max,
    index=[swiss_min, swiss_mean,swiss_max],
    caption='Unemployment rate of Swiss for low unemployment rate of Foreigners'
).add_to(map_)

step = cm.StepColormap(
    [em_swiss_unem_foreign_color, unem_swiss_unem_foreign_color],
    vmin= swiss_min,vmax = swiss_max,
    index=[swiss_min, swiss_mean,swiss_max],
    caption='Unemployment rate of Swiss for high unemployment rate of Foreigners'
).add_to(map_)



print(swiss_min, swiss_mean,swiss_max)
map_